I previously shared a [notebook](url) (see the discussion [here](https://www.kaggle.com/competitions/kaggle-llm-science-exam/discussion/441128)) that found a cluster of relevant Wikipedia STEM articles, resulting in around 270K STEM articles for which the resulting dataset is released [here.](https://www.kaggle.com/datasets/mbanaei/stem-wiki-cohere-no-emb)

However, due to issues with WikiExtractor, there're cases in which some numbers or even paragraphs are missing from the final Wiki parsing. Therefore,  for the same set of  articles, I used Wiki API to gather the articles' contexts (see discussion [here](https://www.kaggle.com/competitions/kaggle-llm-science-exam/discussion/442483)), for which the resulting dataset is released [here](https://www.kaggle.com/datasets/mbanaei/all-paraphs-parsed-expanded).

In order to show that the found articles cover not only the train dataset articles but also a majority of LB gold articles, I release this notebook that uses a simple retrieval model (without any prior indexing) together with a model that is trained only on the RACE dataset. (not fine-tuned on any competition-similar dataset).

The main design choices for the notebook are:
- Using a simple TF-IDF to retrieve contexts from both datasets for every given question.
- Although the majority of high-performing public models use DeBERTa-V3 to do the inference in their pipeline, I used a LongFormer Large model, which enables us to have a much longer prefix context given limited GPU memory. More specifically, as opposed to many public notebooks, there's no splitting to sentence level, and the whole paragraph is retrieved and passed to the classifier as a context (the main reason that we don't get OOM and also have relatively fast inference is that in LongFormer full attention is not computed as opposed to standard models like BERT).
- I use a fall-back model (based on a public notebook that uses an openbook approach and performs 81.5 on LB) that is used for prediction when there's low confidence in the main model's output for the top choice.

P.S: Although the model's performance is relatively good compared to other public notebooks, many design choices can be revised to improve both inference time and performance. (e.g., currently, context retrieval seems to be the inference bottleneck as no prior indexing is used).

In [1]:
# !cp /kaggle/input/datasets-wheel/datasets-2.14.4-py3-none-any.whl /kaggle/working
# !pip install  /kaggle/working/datasets-2.14.4-py3-none-any.whl
# !cp /kaggle/input/backup-806/util_openbook.py .

In [2]:
# # installing offline dependencies
# !pip install -U /kaggle/input/faiss-gpu-173-python310/faiss_gpu-1.7.2-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl
# !cp -rf /kaggle/input/sentence-transformers-222/sentence-transformers /kaggle/working/sentence-transformers
# !pip install -U /kaggle/working/sentence-transformers
# !pip install -U /kaggle/input/blingfire-018/blingfire-0.1.8-py3-none-any.whl

# !pip install --no-index --no-deps /kaggle/input/llm-whls/transformers-4.31.0-py3-none-any.whl
# !pip install --no-index --no-deps /kaggle/input/llm-whls/peft-0.4.0-py3-none-any.whl
# !pip install --no-index --no-deps /kaggle/input/llm-whls/trl-0.5.0-py3-none-any.whl

In [3]:
RUN_ON_KAGGLE = False
DEBUG = False

VAL_SIZE = 100

# file_for_local_cv = "/home/viktor/Documents/kaggle/kaggle_llm/data/kaggle-llm-science-exam/test.csv"
# file_for_local_cv = "/home/viktor/Documents/kaggle/kaggle_llm/data/kaggle-llm-science-exam/train.csv"
# file_for_local_cv = "/home/viktor/Documents/kaggle/kaggle_llm/data/data_dumps/more_questions/more_questions_raw_questions_wiki_sci_3_shuffled.csv"





In [4]:
from util_openbook import get_contexts
import pickle

get_contexts(RUN_ON_KAGGLE, VAL_SIZE, file_for_local_cv)

import gc
gc.collect()


===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please submit your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
CUDA_SETUP: WARNING! libcudart.so not found in any environmental path. Searching /usr/local/cuda/lib64...
CUDA SETUP: CUDA runtime path found: /usr/local/cuda/lib64/libcudart.so
CUDA SETUP: Highest compute capability among GPUs detected: 8.9
CUDA SETUP: Detected CUDA version 121
CUDA SETUP: Loading binary /home/viktor/miniconda3/envs/torch-env/lib/python3.9/site-packages/bitsandbytes/libbitsandbytes_cuda121.so...


/home/viktor/miniconda3/envs/torch-env/lib/python3.9/site-packages/bitsandbytes/cuda_setup/main.py:136: UserWarning: /home/viktor/miniconda3/envs/torch-env did not contain libcudart.so as expected! Searching further paths...
  warn(msg)
/home/viktor/miniconda3/envs/torch-env/lib/python3.9/site-packages/bitsandbytes/cuda_setup/main.py:136: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('vs/workbench/api/node/extensionHostProcess')}
  warn(msg)
/home/viktor/miniconda3/envs/torch-env/lib/python3.9/site-packages/bitsandbytes/cuda_setup/main.py:136: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('module'), PosixPath('//matplotlib_inline.backend_inline')}
  warn(msg)


Batches:   0%|          | 0/7 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/28 [00:00<?, ?it/s]

...

Batches:   0%|          | 0/5584 [00:00<?, ?it/s]

Batches:   0%|          | 0/7 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

54

: 